# ★ Exploration 4. 작사가 인공지능 만들기 ★
## * 아래는 최종 프로젝트에 제출하는 ipynb 노트북 파일입니다. *

### Part 1. 데이터 다듬기 (전처리)

#### 1) 라이브러리 불러오기 및 내용 확인하기

In [1]:
## 필요한 라이브러리를 불러오자.

import glob
import os, re
import tensorflow as tf

txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

## 여러 개의 txt 파일을 모두 읽어서 raw_corpus 에 담자.
## 라인 단위로 끊어서 리스트 형태로 읽어오자.

for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:4])

데이터 크기: 187088
Examples:
 ['At first I was afraid', 'I was petrified', 'I kept thinking I could never live without you', 'By my side But then I spent so many nights']


#### 2) 읽어온 데이터 정제하기

In [2]:
## 정규 표현식을 활용해 아래와 같이 소문자로 바꾸고, 특수문자를 삭제하고 공백을 처리하는 작업을 할 수 있다.

def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4
    sentence = sentence.strip() # 5
    sentence = '<start> ' + sentence + ' <end>' # 6
    return sentence

In [3]:
# 정제된 문장을 모으자.
corpus = []

for sentence in raw_corpus:
    ## 아래와 같이 문장의 요소가 아예 없는 경우이거나 문장이 ':' 로 끝나는 경우는 제외하자.
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
    
    # 정제를 하고 담자.
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
    
# 정제된 결과의 일부를 확인해보자.
corpus[:10]

['<start> at first i was afraid <end>',
 '<start> i was petrified <end>',
 '<start> i kept thinking i could never live without you <end>',
 '<start> by my side but then i spent so many nights <end>',
 '<start> just thinking how you ve done me wrong <end>',
 '<start> i grew strong <end>',
 '<start> i learned how to get along and so you re back <end>',
 '<start> from outer space <end>',
 '<start> i just walked in to find you <end>',
 '<start> here without that look upon your face i should have changed that fucking lock <end>']

#### 3) 토큰화하기

In [4]:
# 토큰 개수가 15개 이상이면 제외를 권장하여, 아래와 같이 제외하자.
corpus_ret = []
for n in range(len(corpus)):
    if corpus[n].count(" ") < 15:
        corpus_ret.append(corpus[n])

print(len(corpus_ret)) # 156013개 출력

156013


In [5]:
# 토큰화 할 때 텐서플로우의 Tokenizer와 pad_sequences를 사용
# 더 잘 알기 위해 아래 문서들을 참고
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/sequence/pad_sequences

# 단어장의 크기는 12,000 이상으로 설정하기를 권장하여 13,000개로 설정하였다.
def tokenize(corpus_ret):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=13000,
        filters=' ',
        oov_token="<unk>"
    )
    # corpus를 이용해 tokenizer 내부의 단어장 완성
    tokenizer.fit_on_texts(corpus_ret)
    # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환
    tensor = tokenizer.texts_to_sequences(corpus_ret)   
    # 입력 데이터의 시퀀스 길이를 일정하게 맞추기
    # 만약 시퀀스가 짧다면 문장 뒤에 패딩을 붙여 길이 맞추기
    # 문장 앞에 패딩을 붙여 길이를 맞추고 싶다면 padding='pre' 사용
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus_ret)

[[   2   70  248 ...    0    0    0]
 [   2    4   53 ...    0    0    0]
 [   2    4 1077 ...    0    0    0]
 ...
 [   2    8    4 ...    0    0    0]
 [   2   44   17 ...    0    0    0]
 [   2    6  172 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7fbb34108410>


In [17]:
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 60: break

1 : <unk>
2 : <start>
3 : <end>
4 : i
5 : ,
6 : the
7 : you
8 : and
9 : a
10 : to
11 : it
12 : me
13 : my
14 : in
15 : that
16 : t
17 : s
18 : on
19 : your
20 : of
21 : we
22 : .
23 : like
24 : m
25 : all
26 : is
27 : be
28 : for
29 : up
30 : with
31 : so
32 : just
33 : but
34 : know
35 : can
36 : love
37 : got
38 : they
39 : what
40 : don
41 : this
42 : no
43 : get
44 : she
45 : when
46 : ?
47 : oh
48 : do
49 : yeah
50 : now
51 : if
52 : baby
53 : was
54 : he
55 : go
56 : re
57 : out
58 : down
59 : one
60 : !


In [7]:
# tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성하자.
# 마지막 토큰은 <end>가 아니라 <pad>일 가능성.
src_input = tensor[:, :-1]  
# tensor에서 <start>를 잘라내서 타겟 문장을 생성하자.
tgt_input = tensor[:, 1:]    

print(src_input[0])
print(tgt_input[0])

[  2  70 248   4  53 708   3   0   0   0   0   0   0   0]
[ 70 248   4  53 708   3   0   0   0   0   0   0   0   0]


### Part 2. 훈련 데이터, 테스트 데이터 분리 및 학습시키기

#### 1) 훈련 데이터, 테스트 데이터 분리하기

In [8]:
from sklearn.model_selection import train_test_split

source = src_input
target = tgt_input
enc_train, enc_val, dec_train, dec_val = train_test_split(source, target, test_size=0.2, shuffle=True, random_state=45)

In [9]:
print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

Source Train: (124810, 14)
Target Train: (124810, 14)


#### 2) tf.keras.Model Subclassing 적용하기

In [10]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

#### 3) 모델 학습시키기

In [11]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

model.compile(loss=loss, optimizer=optimizer)
model.fit(enc_train, dec_train, epochs=6)

Epoch 1/6
3901/3901 [==============================] - 309s 78ms/step - loss: 3.3918
Epoch 2/6
3901/3901 [==============================] - 304s 78ms/step - loss: 2.7381
Epoch 3/6
3901/3901 [==============================] - 304s 78ms/step - loss: 2.4531
Epoch 4/6
3901/3901 [==============================] - 303s 78ms/step - loss: 2.1875
Epoch 5/6
3901/3901 [==============================] - 303s 78ms/step - loss: 1.9603
Epoch 6/6
3901/3901 [==============================] - 304s 78ms/step - loss: 1.7624


### Part 3. 작사가 인공지능 성능 평가하기

In [12]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환하자.
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어를 하나씩 예측해 문장을 만들자.
    #    1. 입력받은 문장의 텐서를 입력하자.
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아내자.
    #    3. 2에서 예측된 word index를 문장 뒤에 붙이자.
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 끝내자.
    while True:
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환해주자. 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

In [21]:
generate_text(model, tokenizer, init_sentence="<start> you make")

'<start> you make me complete <end> '

## ♠ 프로젝트 회고 및 평가 ♠

### 1) 루브릭 평가항목 (노드 4-8)

- A. 가사 텍스트 생성 모델이 정상적으로 동작하는지 : 텍스트 제너레이션 결과가 **그럴듯한 문장**을 만들었는가?
- B. 데이터의 전처리 및 데이터셋 구성 과정이 체계적인지 : **특수문자 제거, 토크나이저 생성, 패딩처리 등의 문자처리 테크닉**을 활용했는가?
- C. 텍스트 생성모델의 학습이 안정적인지 : 생성모델의 validation loss가 **10 epoch 이내 2.2 이하**를 만들어내는가?

### 2) 평가항목에 대한 본인의 결과 및 회고

- A. 해당 주피터 노트북의 코드를 끝까지 돌려서 그럴듯한 문장을 만들어냈습니다. 대충 생성된 문장을 해석하면 '**당신은 나를 완전하게 만들어**' 라는 결과를 얻을 수 있었습니다.
- B. 정규 표현식(Regex)을 활용하여 특수문자 및 공백 처리를 하고 토크나이저 함수를 생성하여 텍스트 생성모델이 필요할 때 적절한 토큰을 사용하게 하여 그럴듯한 문장을 만들어낼 수 있게 할 수 있습니다. 또한 문장의 길이를 균등하게 맞추는 차원에서 패딩을 할 수도 있습니다.
- C. 사실 텍스트 생성모델을 학습시키는 데 시간을 많이 소비했던 것 같습니다. 사실 Exploration 4를 수행하면서 든 의문점에 대해 많은 분들께서 의견을 주셨는데, 가장 인상깊었던 의견은 https://keras.io/api/callbacks/early_stopping/ 링크에 나오는 keras의 callbacks에 기반해 **early_stopping 이라는 메서드를 사용해 원하는 결과가 나올 때 학습을 중단시킴으로서 학습에 걸리는 시간을 단축**시킬 수 있다는 부분이었습니다. 어쨌든 코드상으로는 4 epoch에 2.2 이하로 처음 내려오고 최종 6 epoch에서는 1.7624를 나타내어 해당 목표를 달성할 수 있었습니다!